# Wrangle Report
Devin de Araujo<br>
25.03.2018
<br>
<br>
This document will describe the data wrangling efforts used to produce the **twitter_archive_master.csv**

The report will discuss:
- Key steps
- Errors, limitations or roadblocks encountered

### Download image-predictions.tsv

This was the first step taken. The [Requests]('http://docs.python-requests.org/en/master/') library provided useful tools for programmatically downloading the tsv.

Consideration had to be paid to the file handling method used to open the file type.<br>
- r.content returns binary information so the 'wb' specified the file to be opened for writing in binary mode

<code>with open('image-predictions.tsv', 'wb') as file:<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;file.write(r.content)</code>

### Accessing Tweet info using tweepy

This information required a fair bit of exploration to gather cleanly. I had previously used the [twitter]('https://github.com/sixohsix/twitter) package so wasn't familiar with the [tweepy]('http://www.tweepy.org') package. 

I found it useful to include <code style="color:#4c7b95">wait_on_rate_limit=True</code> and <code style="color:#4c7b95">wait_on_rate_limit_notify=True</code> arguments in the <code>api</code> object. I did not notice any difference in adding the <code>tweet_mode='extended'</code> argument to the tweet object I created, however, I did not spend a great deal of time comparing the results in too much detail. 

<code style="color:#4c7b95">api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)</code><br>
<br>
<code style="color:#4c7b95">tweet = api.get_status('892177421306343426', tweet_mode='extended')</code>


The tweet object (also called *status*) was very messy to work with at the beginning. The object that was returned had no visual JSON structure to it. I managed to access all of the tweet elements I needed but later learnt that there is a private method that can be called on the object to return a nicely formatted <code>JSON</code> object. 

Adding <code style="color:#4c7b95"> _json </code> to the tweet object (<code style="color:#4c7b95">tweet = tweet._json</code>) will do this. 

** Issues **

I spent a great deal of time trying to extract <code style="color:#4c7b95">hashtag</code> information. Not all tweets had hashtags so this meant that I could not set an object called <code style="color:#4c7b95">hashtag</code> to look for a certain index within a dict returned by <code style="color:#4c7b95">tweet.entities['hashtags'][0]['text']</code>. I could grab the whole object with <code style="color:#4c7b95">tweet.entities['hashtags']</code>, which returned a list object <code style="color:#4c7b95"> [{'text': 'BarkWeek', 'indices': [129, 138]}] </code>, containing a dictionary inside, then later extract the <code style="color:#4c7b95">'text'</code> element to get the actual hashtag, but again, I had issues with tweets that had no hashtags so abandoned this. 

## Cleaning the data

My overall goal was to create one master dataframe by combining the twitter-archive and tweet_attributes files. I left the image predictions file out because there were far less records available. I felt that including this in the master file would mean the database as a whole would contain a lot of missing values.

Some challenges around cleaning were faced when cleaning dog names. Pandas compresses rows/columns making it hard to identify <b>all</b> erroneous names. There was also uncertainty around how explicit the <code style="color:#4c7b95">.replace()</code> method is. For example, if I asked it to replace 'the' with a blank string, would it replace a name like 'theo' leaving an 'o' behind. I couldn't find explicit documentation around this issue so cleaned the names as best I could. 


### removing rating and url from text with regex
I was able to easily extract the http links from the text data, but removing the rating proved difficult. I used regular expressions in pandas <code style="color:#4c7b95">.str.extract()</code>. I decided to leave the ratings in because they are part of the actual tweet anyway. If I were going to do any sort of NLP I would investigate this further to remove the ratings.

### Pandas 'melt' function issues

This cause quite a lot of issues. Melting the data frame to produce a tidy format with *'doggo, floofer, pupper, puppo'* columns combined into one column created around 4 x the amount of records, because duplicates were being created for each column, including columns with NaN or blank values. I tried to use the following lambda function I found on [here]('https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-dataframe-in-pandas-python?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa') instead, which worked but I then found that there were some records that contained two types - ie 'doggo' and 'pupper'. Resulting in new categories like **'doggopupper'** or **'doggofloofer'**. 

The code I used with the lambda function is here:
<code style="color:#4c7b95">
<br>
twitter_arch['type'] = twitter_arch[['doggo', 'floofer', 'pupper','puppo']].apply(lambda x : '{}{}{}{}'.format(x[0],x[1],x[2],x[3]), axis=1)
<br>
<br>
twitter_arch['type'] = twitter_arch['type'].str.replace('nan', '')
<br>
twitter_arch['type'].value_counts()
</code>

In the end, I used the melt function, sorted the dataframe by the new **'type'** column and then dropped duplicates, specifying the <code style="color:#4c7b95">keep='first'</code> argument. The other option would have been to clean the 'doggopupper' labels but that would leave duplicate tweet_ids and would mean I was massaging the data. 

I did not have time to complete all of the steps it would have taken to produce a very clean DataFrame, I did not clean any rating numerator or denominator information (until the investigation part), but feel that the quality and tidyness of the final csv I produced is much better than the original data sets provided.